## Классификация 

В данном разделе мы применили к нашим данным классификацию по kNN для того, чтобы понять, какой продукт нужно рекламировать в каком канале. 

In [ ]:
# загружаем библиотеки 
import pandas as pd
import numpy as np
df = pd.read_csv('https://hse.kamran.uz/share/hack2022_df.csv')

Для начала мы хотим понять, с чем мы работаем. 

Описание полей: 
1. Client_id - идентификатор клиента 
2. Gender - пол 
3. Birth_date - дата рождения клиента 
4. Create_dt - дата создания счета/дата создания заявки 
5. nonresident_flag - флаг резидента 
6. businessman_flag - флаг бизнесмена 
7. city - город 
8. term - соглашение об основных условиях сделки 
9. contract_sum - сумма контракта
10. product_category_name - тип продукта (два значения - договор на дк и кк)
11. card_id - идентификатор карты 
12. card_type_name - тип карты 
13. start_date
14. fact_close_date - дата закрытия счет/карты/продукта
15. purchase_sum 
16. purchase_count
17. current_balance_avg_sum
18. current_balance_sum
19. current_debit_turn_sum
20. current_credit_turn_sum
21. card_type

Бинарные категориальные  признаки: 
1. Gender 
2. Nonresident_flg 
3. Businessman_flag 


Категориальные небинарные: 
1. City 
2. product_category_name
3. card_type
4. card_type_name


В ячейках ниже мы обрабатываем наши данные (убираем дубли, преобразуем небинарные признаки в числовые).

In [ ]:
def preprocess(df, ohe_cols=['card_type_name', 'city']):
    df.drop_duplicates(inplace = True)

    del df['term']
    del df['card_id']

    if len(ohe_cols)>1:
        del df['client_id']

    # преобразование небинарных признаков
    one_hot_df = pd.get_dummies(df, 
                                columns=ohe_cols, 
                                drop_first=False)
    
    from datetime import datetime, date
    today = date.today()
    one_hot_df['Year'] = pd.to_datetime(one_hot_df['birth_date'], format='%Y')
    one_hot_df['year'] = pd. DatetimeIndex(one_hot_df['Year']).year
    one_hot_df['age'] = today.year - one_hot_df['year']
    del one_hot_df['Year']
    del one_hot_df['year']
    del one_hot_df['birth_date']

    one_hot_df['life_account'] = one_hot_df['fact_close_date'] - one_hot_df['start_date']
    one_hot_df.loc[one_hot_df["gender"] == "М","gender"] = 1
    one_hot_df.loc[one_hot_df["gender"] == "Ж","gender"] = 0
    one_hot_df.loc[one_hot_df["nonresident_flag"] == "R","nonresident_flag"] = 0
    one_hot_df.loc[one_hot_df["nonresident_flag"] == "N","nonresident_flag"] = 1

    one_hot_df.loc[one_hot_df['card_type'] == "dc","card_type"] = 1
    one_hot_df.loc[one_hot_df['card_type'] == "cc","card_type"] = 0


    one_hot_df.loc[one_hot_df['product_category_name'] == "Кредитная карта","product_category_name"] = 1
    one_hot_df.loc[one_hot_df['product_category_name'] == "Договор на текущий счет для дебетовой карты",'product_category_name'] = 0

    one_hot_df[['start_date', 'fact_close_date']] = np.where(one_hot_df[['start_date', 'fact_close_date']].isnull(), 0, 1)
    one_hot_df['year'] = pd. DatetimeIndex(one_hot_df['create_date']).year
    del one_hot_df['create_date']
    one_hot_df.fillna(0, inplace=True)
    return one_hot_df

def try_different_clusters(K, data):
    from sklearn.cluster import KMeans
    cluster_values = list(range(1, K+1))
    inertias=[]
    clust_models=[]
    
    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)
        clust_models.append(model)
    
    return inertias,clust_models

def fit(one_hot_df):
    from sklearn.cluster import KMeans
    kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)
    kmeans_model.fit(one_hot_df)

    outputs, clust_models = try_different_clusters(7, one_hot_df)
    distances = pd.DataFrame({"clusters": list(range(1, 8)),"sum of squared distances": outputs})

    import plotly.graph_objects as go
    elbow_fig = go.Figure()
    elbow_fig.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

    elbow_fig.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),                  
                    xaxis_title="Количество кластеров",
                    yaxis_title="Сумма расстояний",
                    title_text="Оптимальное количество кластеров")
    
    return elbow_fig, clust_models, distances

In [ ]:
one_hot_df = preprocess(df.copy())

## Нахождение наилучшего продукта для компании в подканале

In [ ]:
ohe_df = preprocess(df.drop(columns='city').copy(), ohe_cols=['card_type_name'])
ohe_df = ohe_df.groupby(['client_id','gender','age','nonresident_flag']).mean().reset_index().drop(columns=['client_id'])

In [ ]:
products = [
       'card_type_name_American Express Optimum',
       'card_type_name_American Express Premier',
       'card_type_name_Eurocard/MasterCard Gold',
       'card_type_name_Eurocard/MasterCard Mass',
       'card_type_name_Eurocard/MasterCard Platinum',
       'card_type_name_Eurocard/MasterCard Virt',
       'card_type_name_Eurocard/MasterCard World',
       'card_type_name_MasterCard Black Edition',
       'card_type_name_MasterCard Electronic',
       'card_type_name_MasterCard World Elite', 
       'card_type_name_MIR Supreme',
       'card_type_name_MIR Privilege Plus',
       'card_type_name_Дебет карта ПС МИР "Бюджетная"',
       'card_type_name_МИР Debit', 'card_type_name_МИР Копилка',
       'card_type_name_МИР СКБ', 'card_type_name_МИР СКБ ЗП',
       'card_type_name_VISA Classic', 'card_type_name_VISA Classic Light',
       'card_type_name_VISA Gold', 'card_type_name_VISA Infinite',
       'card_type_name_VISA Platinum', 'card_type_name_Visa Classic Rewards',
       'card_type_name_Visa Platinum Rewards', 'card_type_name_Visa Rewards',
       'card_type_name_Visa Signature', 
       'card_type_name_Priority Pass',
       ]
product_type = ['American Express']*2+['MasterCard']*8+['MIR']*7+['visa']*9+['Other']*1
product_type = {p:t for p,t in zip(products,product_type)}
user = ['gender','age','nonresident_flag']

In [ ]:
def generate_ds(size=1000,db_size=0.3):
    def make_social_data(num_people, for_age, p_gender, p_res, p_act):
        gender = np.random.choice(binary, num_people, p=[p_gender, 1 - p_gender])
        nonresident_flag = np.random.choice(binary, num_people, p=[p_res, 1 - p_res])
        active = np.random.choice(binary, num_people, p=[p_act, 1 - p_act])
        age = np.random.choice(for_age, num_people)
        
        data_social_m = pd.DataFrame(columns=["gender", "age", "nonresident_flag", "active"])
        data_social_m["gender"], data_social_m["age"], data_social_m["nonresident_flag"], data_social_m["active"] = gender, age, nonresident_flag, active
        
        return data_social_m

    binary = np.arange(2)
    for_age_ = np.arange(65) + 20

    data_credit = make_social_data(int(size*db_size), for_age_, 0.44, 0.9, 0.7)
    data_deb = make_social_data(int(size*(1-db_size)), for_age_, 0.45, 0.9, 0.7)

    data_social_media = pd.concat([data_credit, data_deb], ignore_index=True)
    data_channel = np.random.randint(0,10,int(size*db_size)+int(size*(1-db_size)))
    data_social_media['channel_id'] = data_channel
    return data_social_media

def match_user_product(one_hot_df, user, products, data_social_media):
    from sklearn.neighbors import KNeighborsRegressor
    knrs = [pd.Series(KNeighborsRegressor().fit(one_hot_df[user],one_hot_df[product]).predict(data_social_media.drop(columns='active'))) for product in products]

    ddd = pd.concat(knrs,axis=1)
    ddd.columns = products
    return pd.concat([data_social_media,ddd],axis=1)

data_social_media = generate_ds()
data_social_media

,gender,age,nonresident_flag,active,channel_id
0,0,29,0,0,2
1,1,30,0,0,0
2,1,52,0,0,7
3,1,82,0,1,8
4,0,66,0,0,0
...,...,...,...,...,...
995,1,73,0,1,5
996,0,24,1,0,9
997,1,80,0,1,7
998,1,48,0,0,1


In [ ]:
one_hot_df_ = match_user_product(one_hot_df, user, products, data_social_media.drop(columns='channel_id'))
one_hot_df_['channel_id'] = data_social_media['channel_id']

In [ ]:
px.bar(one_hot_df_.groupby('channel_id').mean(),products)

Данный график отражает по оси OX среднюю уверенность модели в выборе продукта на данном канале. 

In [ ]:
import pickle as pkl
pkl.dump(px.bar(one_hot_df_.groupby('channel_id').mean(),products),open('plt_product.pkl', 'wb'))

## Парсер

In [ ]:
!pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import ast
import pandas as pd
import plotly.express as px
import datetime
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [ ]:
def visits_data(parsed_contents):
    """
    parsed contents = js_script_str
    """
    parsed_contents = re.sub("Highcharts.Map", "Highcharts.Chart", parsed_contents)
    
    # Find the indices of new Highcharts.Chart
    lst_1 = [match.start() for match in re.finditer("new Highcharts.Chart", parsed_contents)][1:]
    lst_2 = [match.end() for match in re.finditer("new Highcharts.Chart", parsed_contents)]
    
    # Pairs of indices of consecutive new Highcharts.Chart to parse everything that's inbetween
    lst_tuples = list(zip(lst_2, lst_1))
    lst_lists = [list(elem) for elem in lst_tuples]
    
    # Adjust the indices to get rid of rubbish
    for t in lst_lists:
        t[0] = t[0] + 1
        t[1] = t[1] - 30
        
    # Extract the contents between the new Highcharts.Chart
    d1_str = parsed_contents[lst_lists[0][0]:lst_lists[0][1]]
    d1_str = re.sub('false', "False", d1_str)
    d1_str = re.sub('null', '""', d1_str)
    re.sub("\\\\",'"', d1_str) ### careful, assignment may be needed!!!
    
    # Convert to dict
    d1 = ast.literal_eval(d1_str)
    
    return d1['chart']['title'], d1['series'][1]['data']

In [ ]:
def web_parse(website):
    root_url = "https://spymetrics.ru/ru/website/" 
    full_url = root_url + website

    # create an HTML Session object
    session = HTMLSession()

    # Use the object above to connect to needed webpage
    html= session.get(full_url).text

    soup = BeautifulSoup(html, "html.parser")
    list_js_scripts = soup.find_all("script", type="text/javascript")
    js_script_str = None
    for js_script in list_js_scripts:
        if "jQuery('#webcompareform')" in str(js_script):
            js_script_str = js_script.contents[0]
            visits = visits_data(js_script_str)

    today = datetime.date.today()
    first = today.replace(day=1)
    prev = [first - i * datetime.timedelta(days=25) for i in range(1,7)]
    monthes = []
    for date in prev:
        monthes.append(date.strftime("%b%Y"))
    monthes = monthes[::-1]

    fig = px.line(y=visits[1], x=monthes, title='Визиты сайта', labels={'x': 'Месяц', 'y': 'Количество посещений'}, color_discrete_sequence=["#4C4C9D"])
    return fig

In [ ]:
web_parse('cian.ru')

Мы спарсили данные с spymetrics. Теперь пользователь может ввести ссылку на сайт и посмотреть его посещаемость. Это помогает маркетолгу проанализировать среду, в которую они собирабтся выходить. Например, если в последний месяц была какая-то аномалия в посещаемости, то вероятно, что на этих пользователях проводится какая-то сторонняя активность. Это дает им дополнительную информацию при принятия решений. 